Token: MTExNjUzMzQ5ODE1NzcyNzg1OA.GOvH6D.Nn8FMZ0PN2_VpbWPuqnSX92iT0esUrfYwbWkL0
URL: https://discord.com/api/oauth2/authorize?client_id=1116533498157727858&permissions=535260822720&scope=bot

In [1]:
import requests
import json
from discord import SyncWebhook
from bs4 import BeautifulSoup

In [2]:
# Define the list of stocks and corresponding purchase price
positions = [
    {"exchange": "NYSE", "symbol": "AMPS", "buyPrice": 5.60},
    {"exchange": "NYSE", "symbol": "ASPN", "buyPrice": 7.20},
    {"exchange": "NASDAQ", "symbol": "AY", "buyPrice": 26.63},
    {"exchange": "NYSE", "symbol": "CHPT", "buyPrice": 9.05},
    {"exchange": "NYSE", "symbol": "CWEN", "buyPrice": 29.74},
    {"exchange": "NASDAQ", "symbol": "NDAQ", "buyPrice": 51.62},
    {"exchange": "NASDAQ", "symbol": "PYPL", "buyPrice": 63.96},
    {"exchange": "NYSE", "symbol": "CI", "buyPrice": 262.13}
]
# stocks w/price watch - send alert if below listed price
watchlist = [
    {"exchange": "NASDAQ", "symbol": "AAPL", "targetPrice": 140.00},
    {"exchange": "NASDAQ", "symbol": "MSFT", "targetPrice": 250.00}
]

In [3]:
alertHook = SyncWebhook.from_url("https://discord.com/api/webhooks/1116536180318687343/R4yarpgveNUL7-ZEEKbZnd8M4KQQoP09OErdU5IDHrl7n9-cOWajfe0j_wKmIBs85OSE")
summaryHook = SyncWebhook.from_url("https://discord.com/api/webhooks/1116776647429996574/UZALIeQWrV_MNQr-xraOGzGjKbxzGU6aWFRnvtFB-tdC1rHhd2oyoauG3Zp3jNY8-Sol")
watchHook = SyncWebhook.from_url("https://discord.com/api/webhooks/1118966208063078451/UnDF6FA8ygVR0_JcMkKl89NQNNnKpOy24_yQysI8hp9O85uuab8LYdSOgbEhXjp8UNLh")
watchSummaryHook = SyncWebhook.from_url("https://discord.com/api/webhooks/1118971294168391770/uzO_9HjiujP4HB1EUY1TknoZq6T4w8kske65LjH0Rh7tK7CBdjHkJe9mFcBEbSJv2Vjv")

In [4]:
for stock in positions:
    output = ""

    # keep stock info in variables for ease of use
    symbol = stock["symbol"]
    exchange = stock["exchange"]
    buyPrice = stock["buyPrice"]

    # get price element
    priceURL = "https://www.google.com/finance/quote/" + symbol + ":" + exchange

    print(priceURL)

    try:
        priceResponse = requests.get(priceURL)
        priceSoup = BeautifulSoup(priceResponse.text, "html.parser")
        priceElement = priceSoup.find('div', class_= 'YMlKec fxKbKc')
        if priceElement == None:
            raise Exception("Element not found.") 
    except requests.HTTPError as e:
        print(f"HTTP Error: {e}")
    except requests.RequestException as e:
        print(f"Error during request: {e}")
    except Exception as e:
        print(f"Error: {e}")

    # get price from element
    priceString = priceElement.get_text()
    priceString = priceString[1:]
    marketPrice = float(priceString)

    # display ticker, market price, purchase price
    output = f"{output}{symbol}        Current Price: **{priceString}**      Buy Price: {str(buyPrice)}"

    # calculate % change      (current price - purchase price / purchase price) * 100
    change = round(((marketPrice - buyPrice) / buyPrice)*100, 2)

    if change > 0:  # if positive, print w/+ sign
        output = f"{output}  **(+{str(change)}%)**\n"
    else:           # else automatically prints w/- sign
        output = f"{output}  **({str(change)}%)**\n"

    # >15% increase alert
    if change > 15:
            alertString = f"{symbol} value is currently **{priceString},** which is higher than 15% increase (+{str(change)}% gain)\n"
            alertHook.send(alertString)

    # create forecast URL w/correct exchange
    if exchange == "NYSE":
        forecastURL = "https://www.wallstreetzen.com/stocks/us/nyse/" + symbol + "/stock-forecast"
    if exchange == "NASDAQ":
        forecastURL = "https://www.wallstreetzen.com/stocks/us/nasdaq/" + symbol + "/stock-forecast"

    output = f"{forecastURL}\n{output}"


    try:        # read HTML from forecast page
        forecastResponse = requests.get(forecastURL)
        forecastSoup = BeautifulSoup(forecastResponse.text, "html.parser")
        forecastElement = forecastSoup.find('div', {'data-react-class' : 'companies/ForecastPage'})
        if forecastElement == None:
            raise Exception("Forecast element not found.")
        forecastDataProps = forecastElement.get('data-react-props')
        forecastProps = json.loads(forecastDataProps)
        analystRatings = forecastProps['analystRatings']
    except requests.HTTPError as e:
        print(f"HTTP Error: {e}")
    except requests.RequestException as e:
        print(f"Error during request: {e}")
    except KeyError as e:
        analystRatings = None
        print(f"Error retrieving analyst ratings: {e}")
    except Exception as e:
        print(f"Error: {e}")

    averageTarget = 0.0
    count = 0.0

    try:
        if analystRatings == None:
            raise Exception("No analyst ratings available.")

        # display each rating
        for rating in analystRatings:

            # store percentile and price target for ease of use
            percentile = rating['analystPercentile']
            target = rating['priceTarget']

            if percentile != None:      # excludes anonymous ratings
                if percentile > 75:     # considers only top 30% of analysts
                    averageTarget += float(target)
                    count += 1.0
                    output = f"{output}{rating['analystName']}           Rating: {rating['ratingText']}          Target: {target}      Top {str(100-float(percentile))}%\n"
        
        if count != 0:      # if count == 0, no top 30% ratings available
            # calculate average price target
            averageTarget = round(averageTarget/count, 2)
            output = f"{output}**Overall: {str(averageTarget)}**\n\n\n"

            # alertLimit = 10% increase of current price would reach target
            alertLimit = averageTarget/marketPrice
            alertLimit = marketPrice*(alertLimit-0.1)

            # if within 10%, send target price alert
            if marketPrice > alertLimit:
                alertString = f"{symbol} value is currently **{priceString},** which is within 10% of target price **{str(averageTarget)}**  (+{str(change)}% gain)\n"
                alertHook.send(alertString)

    except Exception as e:
        print(f"Error: {e}")


    # output stock summary to terminal and discord
    print(output)
    summaryHook.send(output)

for stock in watchlist:
    # keep stock info in variables for ease of use
    symbol = stock["symbol"]
    exchange = stock["exchange"]
    targetPrice = stock["targetPrice"]

    # get price element
    url = "https://www.google.com/finance/quote/" + symbol + ":" + exchange

    print(url)

    try:
        priceResponse = requests.get(url)
        priceSoup = BeautifulSoup(priceResponse.text, "html.parser")
        priceElement = priceSoup.find('div', class_= 'YMlKec fxKbKc')
        if priceElement == None:
            raise Exception("Element not found.") 
    except requests.HTTPError as e:
        print(f"HTTP Error: {e}")
    except requests.RequestException as e:
        print(f"Error during request: {e}")
    except Exception as e:
        print(f"Error: {e}")

    # get price from element
    priceString = priceElement.get_text()
    priceString = priceString[1:]

    if float(priceString) < targetPrice:
        output = f"{symbol} price is currently **{priceString}**, target price is **{str(targetPrice)}**"
        watchHook.send(output)

    # display ticker, market price, purchase price
    output = f"{url}\n{symbol}        Current Price: **{priceString}**      Goal: {str(targetPrice)}"
    watchSummaryHook.send(output)


https://www.google.com/finance/quote/AMPS:NYSE
https://www.wallstreetzen.com/stocks/us/nyse/AMPS/stock-forecast
AMPS        Current Price: **6.11**      Buy Price: 5.6  **(+9.11%)**
Stephen Byrd           Rating: Strong Buy          Target: 10.0      Top 8.0%
James West           Rating: Buy          Target: 10.0      Top 6.0%
**Overall: 10.0**



https://www.google.com/finance/quote/ASPN:NYSE
https://www.wallstreetzen.com/stocks/us/nyse/ASPN/stock-forecast
ASPN        Current Price: **5.83**      Buy Price: 7.2  **(-19.03%)**
Subash Chandra           Rating: Strong Buy          Target: 14.0      Top 3.0%
Alexander Potter           Rating: Strong Buy          Target: 24.0      Top 0.0%
**Overall: 19.0**



https://www.google.com/finance/quote/AY:NASDAQ
https://www.wallstreetzen.com/stocks/us/nasdaq/AY/stock-forecast
AY        Current Price: **21.87**      Buy Price: 26.63  **(-17.87%)**
Stephen Byrd           Rating: Hold          Target: 31.0      Top 8.0%
Shelby Tucker           Rati